<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import os
import sys
sys.path.insert(0, os.path.relpath('../')) 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from super_simulation.super_simulation_factory import SuperSimulationFactory
from transition_table import TransitionTable
from spark_policy import SparkPolicy
from utils.spark_bq_utils import upload_spark_model_inputs
import pandas as pd
from functools import partial

pd.set_option('display.max_rows', 500)

In [4]:
%pwd

'/Users/pacopoler/Desktop/SDC/stanford-decarceration-collective/notebooks'

In [19]:
admissions = pd.read_csv("../state/CA/CA_admissions_by_crime_category.csv")
admissions = admissions.iloc[:, :6]

In [20]:
admissions.head()

,Date,Crimes Against Persons,Property Crimes,Drug Crimes,Other Crimes,Missing
0,January 2018,"1,372",563,256,635,NaN
1,February 2018,"1,405",509,208,582,NaN
2,March 2018,"1,711",621,283,746,NaN
3,April 2018,"1,334",535,222,580,NaN
4,May 2018,"1,832",607,278,745,NaN


In [29]:
total_pop = pd.read_csv("../state/CA/CA_total_pop_by_crime_category.csv")
total_pop = total_pop.iloc[:, :6]
total_pop.head()

,Date,Crimes Against Persons,Property Crimes,Drug Crimes,Other Crimes,Missing
0,January 2018,"98,544","13,755","5,621","11,637",NaN
1,February 2018,"98,444","13,798","5,615","11,546",NaN
2,March 2018,"98,599","13,716","5,628","11,706",NaN
3,April 2018,"98,313","13,617","5,604","11,464",NaN
4,May 2018,"98,465","13,590","5,587","11,677",NaN


In [30]:
# 1. dates to date format
# 2. numbers to int, get rid of commas
# 3. 2020-2021 "missing" column + missing data

In [47]:
month_year = total_pop.Date.str.split(" " , n=1, expand = True)
total_pop["Month"], total_pop["Year"] = month_year.iloc[:, 0], month_year.iloc[:, 1]
total_pop = total_pop.drop("Date", axis = 1)

AttributeError: 'DataFrame' object has no attribute 'Date'

In [48]:
for i in range (len(total_pop)): 
    total_pop.iloc[i,5] = i%12+1

In [ ]:
month_to_number = {
    'January': 1
}

In [50]:
total_pop.Month = total_pop.index % 12 + 1

In [51]:
total_pop.Month = total_pop.Month.map(month_to_number)

,Crimes Against Persons,Property Crimes,Drug Crimes,Other Crimes,Missing,Month,Year
0,"98,544","13,755","5,621","11,637",NaN,1,2018
1,"98,444","13,798","5,615","11,546",NaN,2,2018
2,"98,599","13,716","5,628","11,706",NaN,3,2018
3,"98,313","13,617","5,604","11,464",NaN,4,2018
4,"98,465","13,590","5,587","11,677",NaN,5,2018
5,"98,459","13,627","5,574","11,757",NaN,6,2018
6,"98,397","13,619","5,558","11,736",NaN,7,2018
7,"98,235","13,591","5,522","11,760",NaN,8,2018
8,"98,112","13,540","5,508","11,564",NaN,9,2018
9,"97,950","13,493","5,459","11,670",NaN,10,2018
